In [ ]:
# start coding here
import scanpy as sc
import numpy as np
import pickle
import pandas as pd 
from tqdm.auto import tqdm
from cellwhisperer.utils.processing import ensure_raw_counts_adata

In [ ]:
### loading the geneformer normalization factors
with open(snakemake.input.gene_normalizers, "rb") as fp:
    gene_normalizers = pickle.load(fp)

adata = sc.read_h5ad(snakemake.input.read_count_table)

ensure_raw_counts_adata(adata)

In [ ]:
common_genes = adata.var.reindex(gene_normalizers.keys()).dropna().index
common_gene_symbols = adata.var.gene_name.reindex(common_genes).values

In [ ]:
gene_normalizers = np.exp(np.array([gene_normalizers[key] for key in common_genes]))
gene_normalizers.max()

In [ ]:
adata = adata[:, common_genes].copy()

In [ ]:
top_genes_df = pd.DataFrame(
    index=adata.obs.index,
    data={
        key: pd.Categorical([np.nan] * len(adata), categories=np.unique(common_gene_symbols))
        for key in [f"Top_{i+1}" for i in range(snakemake.params.top_n_genes)]
    }
)
top_genes_df.columns

In [ ]:
for i, obs in enumerate(tqdm(adata.obs.index)):
    cell_data = adata.X[i]
    try:
        cell_data = adata.X[i].toarray().squeeze()
    except AttributeError:
        pass
    normed = cell_data / gene_normalizers
    top_gene_indices = np.argsort(normed)[::-1][:snakemake.params.top_n_genes]

    # Map indices to gene names
    top_gene_names = common_gene_symbols[top_gene_indices] 

    top_genes_df.loc[obs] = top_gene_names

In [ ]:
top_genes_df.to_parquet(snakemake.output.top_genes)

In [ ]:
top_genes_df.iloc[:5]

In [ ]:
top_gene_names[:10]

## Thoughts about `gene_medians`

I intially thought it was weird that all of it operating on "raw" read counts, but couldn't find any evidence for log transforms (in Geneformer paper and code).

Now I think it is fine and 634 might indeed be the maximum median across all genes when looking through 30M cells. But I still have a major concern: The median should take integer values when derived from an raw read count valued dataset.

This honestly suggests to me that most analyzed single cell datasets were log-transformed, which makes this score also log-transformed. Under this interpretation, we should actually take its exponent (probably 2**), before using it to normalize our genes. But then again, there are high values such as 634. Would trimming then be a solution?

On another note: I checked CD19 in a B cell lymphoma (which expresses CD19 according to annotation). Indeed expression levels are strong, yet not strong enough to make it in the top100 genes.

I suspect that the reason for this is that the median value for normalization is non-zero-based (CD19 is expressed in few cells, but strongly, thus the normalizer is also high). I think it would be much better to have a normalizer based on the mean of the log1p across ALL samples (i.e. no zero filtering).

This is what I ended up doing after all